## Is this just magic?  What is Numba doing to make code run quickly?

Short answer: Yes, it's magic.

Long answer: No, it's not magic, but it's a little complicated.  Here's a brief overview.

### Python bytecode

What actually happens when you execute a function written in Python?  It gets parsed and executed by CPython.  
Let's take a trivial example and see what's going on under the hood.

In [ ]:
def add(a, b):
    return a + b

Import the Python disassembler

In [ ]:
import dis

In [ ]:
dis.dis(add)

Makes sense.  Load `a` from the stack, load `b` from the stack, add them together and then return.  Let's see what Numba does in this situation

In [ ]:
from numba import jit

In [ ]:
@jit
def add(a,b):
    return a + b

In [ ]:
dis.dis(add)

Huh?  It's doing nothing?  

Oh wait, we forgot to run it once so it isn't compiled yet.

In [ ]:
add(1., 1.)

In [ ]:
dis.dis(add)

Numba has it's own bytecode-ish equivalent.  Once you have run (compiled) `add` you can access the `inspect_types` method.

In [ ]:
add.inspect_types()

Ok.  Numba is doing some LLVM magic, defining things as `int64` and running smoothly.  

(What happens if you do `add(1., 1.)` and then `inspect_types`?

## Alternatively

In [ ]:
def add_object(a, b):
    return a.x + b.x

In [ ]:
class MyInt(object):
    def __init__(self, x):
        self.x = x

In [ ]:
a = MyInt(5)
b = MyInt(6)

In [ ]:
add_object(a, b)

In [ ]:
dis.dis(add_object)

In [ ]:
add_object_jit = jit(add_object)

In [ ]:
add_object_jit(a, b)

In [ ]:
add_object_jit.inspect_types()

What's all this pyobject business?  

blah blah this means it has been compiled in `object` mode.  might be a _little_ faster than regular python if it can do loop lifting, but generally no benefit (except compatibility).  
we want those `pyobjects` to be `int64` or equivalent type which means forcing `nopython` mode

## introduce @jit(nopython=True) and @njit (shortcut for previous)

Oooh -> from here to test notebook where people can try out `nopython=True` on an un-jittable function.  This can lead into n-body stuff using numpy dtypes